https://www.kaggle.com/beezus666/end-to-end-data-wrangling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import regex as re
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
import gc
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype

## Helper functions
- These helper and data cleaning functions are from the old fast.ai course
- The repository is here: https://github.com/fastai/fastai/tree/master/old

In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(df)

        

def make_date(df, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)

        
        
def add_datepart(df, fldnames, drop=True, time=False, errors="raise"):
    # add_datepart converts a column of df from a datetime64 to many columns containing the information from the date. 
    # This applies changes inplace.
    if isinstance(fldnames,str): 
        fldnames = [fldnames]
    for fldname in fldnames:
        fld = df[fldname]
        fld_dtype = fld.dtype
        if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            fld_dtype = np.datetime64

        if not np.issubdtype(fld_dtype, np.datetime64):
            df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
        targ_pre = re.sub('[Dd]ate$', '', fldname)
        attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
                'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
        if time: attr = attr + ['Hour', 'Minute', 'Second']
        for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
        df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
        if drop: df.drop(fldname, axis=1, inplace=True)
            
            
            
def ifnone (a,b): #(a:Any,b:Any)->Any:
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a


# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


def train_cats(df):    
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

            
            
def apply_cats(df, trn):
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = c.astype('category').cat.as_ordered()
            df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)


            
def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

        

def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)      

In [3]:
# This function I believe came from this guy: https://www.kaggle.com/siavrez

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    #return df

In [4]:
df_cal = pd.read_csv('dataset/calendar.csv', parse_dates = ['date'])
df_sales_train = pd.read_csv('dataset/sales_train_validation.csv')
df_prices = pd.read_csv('dataset//sell_prices.csv')
df_submissions = pd.read_csv('dataset//sample_submission.csv')

In [5]:
df_cal.shape, df_sales_train.shape, df_prices.shape, df_submissions.shape

((1969, 14), (30490, 1919), (6841121, 4), (60980, 29))

In [6]:
df_sales_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


**Melt** The pandas melt method unpivots column values and puts it in rows... i.e. melting creates a row for every day for an item/store combination. Doing this puts the data in a flat format that in a few cells will allow us us to merge the sales training data with the calendar and pricing data sets.   
**Columns --> Rows**

In [7]:
# Get all columns that aren't day columns
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df_melted_sales  = df_sales_train.melt(id_vars= id_cols,
                               var_name= 'd',
                               value_name = 'sales')

In [8]:
df_melted_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [9]:
# Dropping redundant columns 
df_melted_sales.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1, inplace = True) 
df_melted_sales.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1,0
1,HOBBIES_1_002_CA_1_validation,d_1,0
2,HOBBIES_1_003_CA_1_validation,d_1,0
3,HOBBIES_1_004_CA_1_validation,d_1,0
4,HOBBIES_1_005_CA_1_validation,d_1,0


In [10]:
df_submissions.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
sub_cols = df_submissions.drop(['id'], axis=1).columns
print(sub_cols)

Index(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')


In [12]:
# Like what we did to the sales data, we also need to melt the submission 
df_melted_sub = df_submissions.melt(id_vars = ['id'], value_vars = sub_cols, 
                                    var_name = 'd', value_name = 'sales') 
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,F1,0
1,HOBBIES_1_002_CA_1_validation,F1,0
2,HOBBIES_1_003_CA_1_validation,F1,0
3,HOBBIES_1_004_CA_1_validation,F1,0
4,HOBBIES_1_005_CA_1_validation,F1,0


In [13]:
df_melted_sub['d'] = df_melted_sub['d'].str.replace('F','')
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1,0
1,HOBBIES_1_002_CA_1_validation,1,0
2,HOBBIES_1_003_CA_1_validation,1,0
3,HOBBIES_1_004_CA_1_validation,1,0
4,HOBBIES_1_005_CA_1_validation,1,0


In [14]:
df_melted_sub['d'] = pd.to_numeric(df_melted_sub['d'], errors='coerce') 
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1,0
1,HOBBIES_1_002_CA_1_validation,1,0
2,HOBBIES_1_003_CA_1_validation,1,0
3,HOBBIES_1_004_CA_1_validation,1,0
4,HOBBIES_1_005_CA_1_validation,1,0


In [15]:
df_melted_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1707440 entries, 0 to 1707439
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      object
 1   d       int64 
 2   sales   int64 
dtypes: int64(2), object(1)
memory usage: 39.1+ MB


In [16]:
df_melted_sub['d'] = df_melted_sub['d'] + 1913
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1914,0
1,HOBBIES_1_002_CA_1_validation,1914,0
2,HOBBIES_1_003_CA_1_validation,1914,0
3,HOBBIES_1_004_CA_1_validation,1914,0
4,HOBBIES_1_005_CA_1_validation,1914,0


In [17]:
df_melted_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1707440 entries, 0 to 1707439
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      object
 1   d       int64 
 2   sales   int64 
dtypes: int64(2), object(1)
memory usage: 39.1+ MB


In [18]:
df_melted_sub

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1914,0
1,HOBBIES_1_002_CA_1_validation,1914,0
2,HOBBIES_1_003_CA_1_validation,1914,0
3,HOBBIES_1_004_CA_1_validation,1914,0
4,HOBBIES_1_005_CA_1_validation,1914,0
...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,1941,0
1707436,FOODS_3_824_WI_3_evaluation,1941,0
1707437,FOODS_3_825_WI_3_evaluation,1941,0
1707438,FOODS_3_826_WI_3_evaluation,1941,0


In [19]:
df_melted_sub = df_melted_sub.applymap(str)
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1914,0
1,HOBBIES_1_002_CA_1_validation,1914,0
2,HOBBIES_1_003_CA_1_validation,1914,0
3,HOBBIES_1_004_CA_1_validation,1914,0
4,HOBBIES_1_005_CA_1_validation,1914,0


In [20]:
df_melted_sub['d'] = 'd_'+ df_melted_sub['d'].astype(str)
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1914,0
1,HOBBIES_1_002_CA_1_validation,d_1914,0
2,HOBBIES_1_003_CA_1_validation,d_1914,0
3,HOBBIES_1_004_CA_1_validation,d_1914,0
4,HOBBIES_1_005_CA_1_validation,d_1914,0


In [21]:
df_melted_sub.tail() 

,id,d,sales
1707435,FOODS_3_823_WI_3_evaluation,d_1941,0
1707436,FOODS_3_824_WI_3_evaluation,d_1941,0
1707437,FOODS_3_825_WI_3_evaluation,d_1941,0
1707438,FOODS_3_826_WI_3_evaluation,d_1941,0
1707439,FOODS_3_827_WI_3_evaluation,d_1941,0


In [22]:
# lists all dataframes we have 
%who DataFrame

df_cal	 df_melted_sales	 df_melted_sub	 df_prices	 df_sales_train	 df_submissions	 


In [23]:
del df_sales_train
del df_submissions

gc.collect()

88

In [24]:
reduce_mem_usage(df_cal)
reduce_mem_usage(df_melted_sales)
reduce_mem_usage(df_melted_sub) 
reduce_mem_usage(df_prices)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 1001.26 Mb (25.0% reduction)
Mem. usage decreased to 39.08 Mb (0.0% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [29]:
del df_test, df_train

In [30]:
df_melted_sales.shape

(58327370, 3)

### Split data: train and test

In [31]:
# Doing it as an 80/20 split as is standard for train/test.
df_test = df_melted_sales.tail(20000) 
df_train = df_melted_sales.iloc[-120000:-20000] 
df_submission = df_melted_sub.copy() # Copied just to keep naming conventions consistent

In [34]:
df_train.tail() 

,id,d,sales
58307365,HOUSEHOLD_2_243_CA_4_validation,d_1913,0
58307366,HOUSEHOLD_2_244_CA_4_validation,d_1913,0
58307367,HOUSEHOLD_2_245_CA_4_validation,d_1913,0
58307368,HOUSEHOLD_2_246_CA_4_validation,d_1913,0
58307369,HOUSEHOLD_2_247_CA_4_validation,d_1913,3


In [35]:
df_test.head()

,id,d,sales
58307370,HOUSEHOLD_2_248_CA_4_validation,d_1913,2
58307371,HOUSEHOLD_2_249_CA_4_validation,d_1913,0
58307372,HOUSEHOLD_2_250_CA_4_validation,d_1913,0
58307373,HOUSEHOLD_2_251_CA_4_validation,d_1913,0
58307374,HOUSEHOLD_2_252_CA_4_validation,d_1913,0


In [36]:
df_train.shape, df_test.shape, df_submission.shape

((100000, 3), (20000, 3), (1707440, 3))

In [37]:
df_cal.shape

(1969, 14)

### Merge

In [38]:
# Merge the calendar data with the training, test and submission DFs
df_train = df_train.merge(df_cal, left_on= 'd', right_on= 'd', how = 'left')
df_test = df_test.merge(df_cal, left_on= 'd', right_on= 'd', how = 'left')
df_submission = df_submission.merge(df_cal, left_on='d', right_on='d', how='left')

df_train.shape, df_test.shape, df_submission.shape # should be same numb of cols

((100000, 16), (20000, 16), (1707440, 16))

In [39]:
df_train.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,FOODS_2_134_CA_1_validation,d_1910,3,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0
1,FOODS_2_135_CA_1_validation,d_1910,6,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0
2,FOODS_2_136_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0
3,FOODS_2_137_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0
4,FOODS_2_138_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0


In [40]:
df_test.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOUSEHOLD_2_248_CA_4_validation,d_1913,2,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
1,HOUSEHOLD_2_249_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
2,HOUSEHOLD_2_250_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
3,HOUSEHOLD_2_251_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0
4,HOUSEHOLD_2_252_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0


In [41]:
df_submission.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0


In [42]:
df_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [43]:
# Because we dropped columns from the training/test/submit DFs, we have to do this to be able to join to prices.
df_prices['id'] = df_prices['item_id'] +'_' + df_prices['store_id']
df_prices.drop(['item_id', 'store_id'], axis=1, inplace = True)
df_prices.head()

,wm_yr_wk,sell_price,id
0,11325,9.578125,HOBBIES_1_001_CA_1
1,11326,9.578125,HOBBIES_1_001_CA_1
2,11327,8.257812,HOBBIES_1_001_CA_1
3,11328,8.257812,HOBBIES_1_001_CA_1
4,11329,8.257812,HOBBIES_1_001_CA_1


In [44]:
df_prices.shape

(6841121, 3)

In [45]:
# The training and submission data have a suffix that's not in the price data.
# So here we're creating a column that matches the price id column.

df_train['id_for_price'] = df_train['id'].str.replace('_validation','')
df_test['id_for_price'] = df_test['id'].str.replace('_validation','')
df_submission['id_for_price'] = df_submission['id'].str.replace('_evaluation','')
df_submission['id_for_price'] = df_submission['id_for_price'].str.replace('_validation','')

In [46]:
df_train.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id_for_price
0,FOODS_2_134_CA_1_validation,d_1910,3,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_134_CA_1
1,FOODS_2_135_CA_1_validation,d_1910,6,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_135_CA_1
2,FOODS_2_136_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_136_CA_1
3,FOODS_2_137_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_137_CA_1
4,FOODS_2_138_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_138_CA_1


In [47]:
df_test.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id_for_price
0,HOUSEHOLD_2_248_CA_4_validation,d_1913,2,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOUSEHOLD_2_248_CA_4
1,HOUSEHOLD_2_249_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOUSEHOLD_2_249_CA_4
2,HOUSEHOLD_2_250_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOUSEHOLD_2_250_CA_4
3,HOUSEHOLD_2_251_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOUSEHOLD_2_251_CA_4
4,HOUSEHOLD_2_252_CA_4_validation,d_1913,0,2016-04-24,11613,Sunday,2,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOUSEHOLD_2_252_CA_4


In [48]:
df_submission.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id_for_price
0,HOBBIES_1_001_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_001_CA_1
1,HOBBIES_1_002_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_002_CA_1
2,HOBBIES_1_003_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_003_CA_1
3,HOBBIES_1_004_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_004_CA_1
4,HOBBIES_1_005_CA_1_validation,d_1914,0,2016-04-25,11613,Monday,3,4,2016,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_005_CA_1


In [49]:
df_submission.tail()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id_for_price
1707435,FOODS_3_823_WI_3_evaluation,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_3_823_WI_3
1707436,FOODS_3_824_WI_3_evaluation,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_3_824_WI_3
1707437,FOODS_3_825_WI_3_evaluation,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_3_825_WI_3
1707438,FOODS_3_826_WI_3_evaluation,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_3_826_WI_3
1707439,FOODS_3_827_WI_3_evaluation,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_3_827_WI_3


In [50]:
# Merge the price data with the training, test and submission DFs
df_train = pd.merge(df_train, df_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], right_on = ['id', 'wm_yr_wk'])
df_test = pd.merge(df_test, df_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'],   right_on = ['id', 'wm_yr_wk'])
df_submission = pd.merge(df_submission, df_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], right_on = ['id', 'wm_yr_wk'])


In [51]:
# Check to see if there are any missing prices
df_train['sell_price'].count(), df_train.shape, df_test['sell_price'].count(), df_test.shape, df_submission['sell_price'].count(), df_submission.shape

(100000, (100000, 19), 20000, (20000, 19), 1707440, (1707440, 19))

In [52]:
df_train.head()

,id_x,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id_for_price,sell_price,id_y
0,FOODS_2_134_CA_1_validation,d_1910,3,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_134_CA_1,2.980469,FOODS_2_134_CA_1
1,FOODS_2_135_CA_1_validation,d_1910,6,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_135_CA_1,2.480469,FOODS_2_135_CA_1
2,FOODS_2_136_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_136_CA_1,2.869141,FOODS_2_136_CA_1
3,FOODS_2_137_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_137_CA_1,2.500000,FOODS_2_137_CA_1
4,FOODS_2_138_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,FOODS_2_138_CA_1,1.839844,FOODS_2_138_CA_1


In [53]:
# Merging with price added a bunch of unneeded columns
df_train.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_train.rename(columns = {"id_x":"id"}, inplace = True)
df_test.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_test.rename(columns = {"id_x":"id"}, inplace = True)
df_submission.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_submission.rename(columns = {"id_x":"id"}, inplace = True)

In [54]:
%who DataFrame

df_cal	 df_melted_sales	 df_melted_sub	 df_prices	 df_submission	 df_test	 df_train	 


In [55]:
# To save some RAM we will get rid of the dataframes that we merged from and are not in use now.
del df_cal
del df_melted_sales
del df_melted_sub
del df_prices
gc.collect()

13332

In [56]:
reduce_mem_usage(df_train)
reduce_mem_usage(df_test)
reduce_mem_usage(df_submission)

Mem. usage decreased to  8.11 Mb (0.0% reduction)
Mem. usage decreased to  1.60 Mb (1.2% reduction)
Mem. usage decreased to 148.18 Mb (0.0% reduction)


In [57]:
# Pickle seems ok though
df_train.to_pickle('df_train_pickle.pkl')
df_test.to_pickle('df_test_pickle.pkl')
df_submission.to_pickle('df_submissions_pickle.pkl')

In [58]:
df_train.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_2_134_CA_1_validation,d_1910,3,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.980469
1,FOODS_2_135_CA_1_validation,d_1910,6,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.480469
2,FOODS_2_136_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.869141
3,FOODS_2_137_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.500000
4,FOODS_2_138_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.839844


In [ ]:
# Uncomment to load the pickle files...
#df_train = pd.read_pickle('df_train_pickle.pkl')
#df_test = pd.read_pickle('df_test_pickle.pkl')
#df_submission = pd.read_pickle('df_submissions_pickle.pkl')

### Transform
we'll do some transforms to get all the data in the DFs numeric and remove nulls

In [59]:
# list column names that have missing values in them
df_train.columns[df_train.isnull().any()]

Index(['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], dtype='object')

In [60]:
# Before removing nulls we add a boolean column to declare if the value was missing
cols_with_missing = df_train.columns[df_train.isnull().any()].tolist()

for col in cols_with_missing:
    df_train[col + '_was_missing'] = df_train[col].isnull()
    df_test[col + '_was_missing'] = df_test[col].isnull()
    df_submission[col + '_was_missing'] = df_submission[col].isnull()

In [61]:
df_train.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,...,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing
0,FOODS_2_134_CA_1_validation,d_1910,3,2016-04-21,11612,Thursday,6,4,2016,NaN,...,NaN,NaN,0,0,0,2.980469,True,True,True,True
1,FOODS_2_135_CA_1_validation,d_1910,6,2016-04-21,11612,Thursday,6,4,2016,NaN,...,NaN,NaN,0,0,0,2.480469,True,True,True,True
2,FOODS_2_136_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,...,NaN,NaN,0,0,0,2.869141,True,True,True,True
3,FOODS_2_137_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,...,NaN,NaN,0,0,0,2.500000,True,True,True,True
4,FOODS_2_138_CA_1_validation,d_1910,0,2016-04-21,11612,Thursday,6,4,2016,NaN,...,NaN,NaN,0,0,0,1.839844,True,True,True,True


In [62]:
imputer_values = {'event_name_1' :'None', 'event_type_1' :'None', 
                  'event_name_2' :'None', 'event_type_2' :'None'                  
                 }
df_train.fillna(value = imputer_values, inplace = True)
df_test.fillna(value = imputer_values, inplace = True)
df_submission.fillna(value = imputer_values, inplace = True)

In [63]:
# Checking to see if there are any missing values
# The no of rows in each DF should be equal to the count
df_train.shape, df_train.count(), df_test.shape, df_test.count(), df_submission.shape, df_submission.count()

((100000, 21),
 id                          100000
 d                           100000
 sales                       100000
 date                        100000
 wm_yr_wk                    100000
 weekday                     100000
 wday                        100000
 month                       100000
 year                        100000
 event_name_1                100000
 event_type_1                100000
 event_name_2                100000
 event_type_2                100000
 snap_CA                     100000
 snap_TX                     100000
 snap_WI                     100000
 sell_price                  100000
 event_name_1_was_missing    100000
 event_type_1_was_missing    100000
 event_name_2_was_missing    100000
 event_type_2_was_missing    100000
 dtype: int64,
 (20000, 21),
 id                          20000
 d                           20000
 sales                       20000
 date                        20000
 wm_yr_wk                    20000
 weekday                 

In [64]:
#list of all non-numeric data
df_train.select_dtypes(include = 'object').columns 

Index(['id', 'd', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2'],
      dtype='object')

In [65]:
#we're going to use pandas date exploding features, so the given dates will wind up being redundant
drop_fields = ['weekday', 'year', 'wday', 'month']
df_train.drop(drop_fields, axis = 1, inplace = True)
df_test.drop(drop_fields, axis = 1, inplace = True)
df_submission.drop(drop_fields, axis = 1, inplace = True)
df_train.shape, df_test.shape, df_submission.shape

((100000, 17), (20000, 17), (1707440, 17))

In [66]:
df_submission.head()

,id,d,sales,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing
0,HOBBIES_1_001_CA_1_validation,d_1914,0,2016-04-25,11613,None,None,None,None,0,0,0,8.382812,True,True,True,True
1,HOBBIES_1_002_CA_1_validation,d_1914,0,2016-04-25,11613,None,None,None,None,0,0,0,3.970703,True,True,True,True
2,HOBBIES_1_003_CA_1_validation,d_1914,0,2016-04-25,11613,None,None,None,None,0,0,0,2.970703,True,True,True,True
3,HOBBIES_1_004_CA_1_validation,d_1914,0,2016-04-25,11613,None,None,None,None,0,0,0,4.640625,True,True,True,True
4,HOBBIES_1_005_CA_1_validation,d_1914,0,2016-04-25,11613,None,None,None,None,0,0,0,2.880859,True,True,True,True


In [68]:
df_submission.tail()

,id,d,sales,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing
1707435,FOODS_3_823_WI_3_evaluation,d_1941,0,2016-05-22,11617,None,None,None,None,0,0,0,2.980469,True,True,True,True
1707436,FOODS_3_824_WI_3_evaluation,d_1941,0,2016-05-22,11617,None,None,None,None,0,0,0,2.480469,True,True,True,True
1707437,FOODS_3_825_WI_3_evaluation,d_1941,0,2016-05-22,11617,None,None,None,None,0,0,0,3.980469,True,True,True,True
1707438,FOODS_3_826_WI_3_evaluation,d_1941,0,2016-05-22,11617,None,None,None,None,0,0,0,1.280273,True,True,True,True
1707439,FOODS_3_827_WI_3_evaluation,d_1941,0,2016-05-22,11617,None,None,None,None,0,0,0,1.000000,True,True,True,True


In [67]:
# The next cell will change the IDs into categorical values.
# But the submission file needs them in the original format, so saving them to a series for use below
df_sub_ids = df_submission['id'].copy()

In [69]:
# We'll need the d column for the submission file format.
# But we can drop it from the train/test dfs as we already have date information in the DF
df_train.drop(['d'], axis = 1, inplace = True)
df_test.drop(['d'], axis = 1, inplace = True)
df_sub_d = df_submission.pop('d')

In [70]:
# Label encode
# For RAM considerations, we're not going to one hot encode anything.
# These are functions from fast.ai that convert objects to categories and keeps those categories consistent across training and test
train_cats(df_train)
apply_cats(df_test, df_train)
apply_cats(df_submission, df_train)
df_train.shape, df_test.shape, df_submission.shape

((100000, 16), (20000, 16), (1707440, 16))

In [71]:
# Change categories to numbers
cat_cols = df_train.select_dtypes(include = 'category').columns
for i in cat_cols:
    df_train['cat_'+i] = df_train[i].cat.codes
    df_test['cat_'+i] = df_test[i].cat.codes
    df_submission['cat_'+i] = df_submission[i].cat.codes


In [72]:
df_train.drop(cat_cols, axis = 1, inplace = True)
df_test.drop(cat_cols, axis = 1, inplace = True)
df_submission.drop(cat_cols, axis = 1, inplace = True)

In [72]:
# Should return an empty list
df_train.select_dtypes(include = 'category').columns 

Index([], dtype='object')

In [73]:
# Should return an empty list
df_train.select_dtypes(include = 'object').columns

Index([], dtype='object')

In [74]:
# Explode out the date column into a ton of columns that a model can use
date_cols = df_train.select_dtypes(include = 'datetime64').columns
for i in date_cols:
    add_datepart(df_train, i)
    add_datepart(df_test, i)
    add_datepart(df_submission, i)

In [75]:
# Should see many more columns at the end of the list (year, month... is_year_end, etc) 
# Should also not see a "date" column
df_train.columns

Index(['sales', 'wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
       'event_name_1_was_missing', 'event_type_1_was_missing',
       'event_name_2_was_missing', 'event_type_2_was_missing', 'cat_id',
       'cat_event_name_1', 'cat_event_type_1', 'cat_event_name_2',
       'cat_event_type_2', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [76]:
# should only see number and boolean columns
df_train.dtypes 

sales                         int16
wm_yr_wk                      int16
snap_CA                        int8
snap_TX                        int8
snap_WI                        int8
sell_price                  float16
event_name_1_was_missing       bool
event_type_1_was_missing       bool
event_name_2_was_missing       bool
event_type_2_was_missing       bool
cat_id                        int16
cat_event_name_1               int8
cat_event_type_1               int8
cat_event_name_2               int8
cat_event_type_2               int8
Year                          int64
Month                         int64
Week                          int64
Day                           int64
Dayofweek                     int64
Dayofyear                     int64
Is_month_end                   bool
Is_month_start                 bool
Is_quarter_end                 bool
Is_quarter_start               bool
Is_year_end                    bool
Is_year_start                  bool
Elapsed                     

In [77]:
reduce_mem_usage(df_train)
reduce_mem_usage(df_test)
reduce_mem_usage(df_submission)

Mem. usage decreased to  4.20 Mb (50.6% reduction)
Mem. usage decreased to  0.82 Mb (51.1% reduction)
Mem. usage decreased to 83.05 Mb (46.3% reduction)


In [78]:
gc.collect()

2533

In [79]:
df_train.head()

,sales,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,3,11612,0,0,0,2.980469,True,True,True,True,...,21,3,112,False,False,False,False,False,False,1461196800
1,6,11612,0,0,0,2.480469,True,True,True,True,...,21,3,112,False,False,False,False,False,False,1461196800
2,0,11612,0,0,0,2.869141,True,True,True,True,...,21,3,112,False,False,False,False,False,False,1461196800
3,0,11612,0,0,0,2.500000,True,True,True,True,...,21,3,112,False,False,False,False,False,False,1461196800
4,0,11612,0,0,0,1.839844,True,True,True,True,...,21,3,112,False,False,False,False,False,False,1461196800


In [80]:
df_test.head()

,sales,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2,11613,0,0,0,8.968750,True,True,True,True,...,24,6,115,False,False,False,False,False,False,1461456000
1,0,11613,0,0,0,8.640625,True,True,True,True,...,24,6,115,False,False,False,False,False,False,1461456000
2,0,11613,0,0,0,6.960938,True,True,True,True,...,24,6,115,False,False,False,False,False,False,1461456000
3,0,11613,0,0,0,3.970703,True,True,True,True,...,24,6,115,False,False,False,False,False,False,1461456000
4,0,11613,0,0,0,6.968750,True,True,True,True,...,24,6,115,False,False,False,False,False,False,1461456000


In [81]:
df_submission.head()

,sales,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,11613,0,0,0,8.382812,True,True,True,True,...,25,0,116,False,False,False,False,False,False,1461542400
1,0,11613,0,0,0,3.970703,True,True,True,True,...,25,0,116,False,False,False,False,False,False,1461542400
2,0,11613,0,0,0,2.970703,True,True,True,True,...,25,0,116,False,False,False,False,False,False,1461542400
3,0,11613,0,0,0,4.640625,True,True,True,True,...,25,0,116,False,False,False,False,False,False,1461542400
4,0,11613,0,0,0,2.880859,True,True,True,True,...,25,0,116,False,False,False,False,False,False,1461542400


In [82]:
# train and test we'll use below, the submission file sales column is all 0's so, we can just get rid of it.
y_train = df_train.pop('sales')
y_test = df_test.pop('sales')
df_submission.drop(['sales'], axis =1, inplace = True)

In [83]:
# As noted in the intro we're just using an out of the box RF for now.a
m = RandomForestRegressor(n_jobs =-1) 
%time m.fit(df_train, y_train)

Wall time: 10 s


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [84]:
# Scoring in r-squared because it's the default and this is the quick start notebook
# r-squared over 0 means that we're predicting better than if we just predicted the average value
# r-squared of 1 means perfect
# it's a good enough metric to optimize the model on
m.score (df_test, y_test)

0.27266814412295926

In [85]:
# For making predictions that we'll submit put train and test together and fit the model again for better predictions
train_test_concat = pd.concat([df_test, df_train])
y_concat = pd.concat([y_test, y_train])
m.fit(train_test_concat, y_concat)

# Scores much better on values it's seen! if only the world were so easy...
m.score (train_test_concat, y_concat)

0.947416588182486

In [86]:
df_submission.dtypes

wm_yr_wk                      int16
snap_CA                        int8
snap_TX                        int8
snap_WI                        int8
sell_price                  float16
event_name_1_was_missing       bool
event_type_1_was_missing       bool
event_name_2_was_missing       bool
event_type_2_was_missing       bool
cat_id                        int16
cat_event_name_1               int8
cat_event_type_1               int8
cat_event_name_2               int8
cat_event_type_2               int8
Year                          int16
Month                          int8
Week                           int8
Day                            int8
Dayofweek                      int8
Dayofyear                     int16
Is_month_end                   bool
Is_month_start                 bool
Is_quarter_end                 bool
Is_quarter_start               bool
Is_year_end                    bool
Is_year_start                  bool
Elapsed                       int32
dtype: object

In [87]:
# And finally, let's make some predictions that we'll submit
predictions = m.predict(df_submission)
predictions

array([1.05, 0.09, 0.78, ..., 1.28, 0.89, 1.75])

In [88]:
# Add the predictions made above and the original ids back onto the dataframe
df_submission['sales'] = predictions
df_submission['id'] = df_sub_ids
df_submission['F'] = df_sub_d

In [89]:
df_submission.head()

,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,event_name_1_was_missing,event_type_1_was_missing,event_name_2_was_missing,event_type_2_was_missing,cat_id,...,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,sales,id,F
0,11613,0,0,0,8.382812,True,True,True,True,14370,...,False,False,False,False,False,False,1461542400,1.05,HOBBIES_1_001_CA_1_validation,d_1914
1,11613,0,0,0,3.970703,True,True,True,True,14380,...,False,False,False,False,False,False,1461542400,0.09,HOBBIES_1_002_CA_1_validation,d_1914
2,11613,0,0,0,2.970703,True,True,True,True,14390,...,False,False,False,False,False,False,1461542400,0.78,HOBBIES_1_003_CA_1_validation,d_1914
3,11613,0,0,0,4.640625,True,True,True,True,14400,...,False,False,False,False,False,False,1461542400,3.45,HOBBIES_1_004_CA_1_validation,d_1914
4,11613,0,0,0,2.880859,True,True,True,True,14410,...,False,False,False,False,False,False,1461542400,2.79,HOBBIES_1_005_CA_1_validation,d_1914


In [90]:
d = {'id': df_sub_ids,
     'F': df_sub_d, 
     'preds': predictions}
sub_file = pd.DataFrame(data=d)
sub_file.head()

,id,F,preds
0,HOBBIES_1_001_CA_1_validation,d_1914,1.05
1,HOBBIES_1_002_CA_1_validation,d_1914,0.09
2,HOBBIES_1_003_CA_1_validation,d_1914,0.78
3,HOBBIES_1_004_CA_1_validation,d_1914,3.45
4,HOBBIES_1_005_CA_1_validation,d_1914,2.79


In [91]:
# As per the competition requirements, we have to change the "d" values to F values
sub_file['F'] = sub_file['F'].str.replace('d_','')
sub_file['F'] = pd.to_numeric(sub_file['F'], errors='coerce') 
sub_file['F'] = sub_file['F'] - 1913
sub_file['F'] = 'F'+ sub_file['F'].astype(str)
sub_file.head()

,id,F,preds
0,HOBBIES_1_001_CA_1_validation,F1,1.05
1,HOBBIES_1_002_CA_1_validation,F1,0.09
2,HOBBIES_1_003_CA_1_validation,F1,0.78
3,HOBBIES_1_004_CA_1_validation,F1,3.45
4,HOBBIES_1_005_CA_1_validation,F1,2.79


In [92]:
# Pivot and reset the index to flatten the file
sub_file = sub_file.pivot(index='id', columns='F', values='preds')
sub_file = sub_file.reset_index()
sub_file.head()

F,id,F1,F10,F11,F12,F13,F14,F15,F16,F17,...,F26,F27,F28,F3,F4,F5,F6,F7,F8,F9
0,FOODS_1_001_CA_1_evaluation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
1,FOODS_1_001_CA_1_validation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
2,FOODS_1_001_CA_2_evaluation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
3,FOODS_1_001_CA_2_validation,1.95,1.37,1.37,1.29,1.40,1.96,1.37,1.37,1.37,...,1.29,1.40,1.88,1.95,1.95,1.91,2.14,1.96,1.37,1.37
4,FOODS_1_001_CA_3_evaluation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08


In [93]:
sub_file.shape

(60980, 29)

In [94]:
sub_file

F,id,F1,F10,F11,F12,F13,F14,F15,F16,F17,...,F26,F27,F28,F3,F4,F5,F6,F7,F8,F9
0,FOODS_1_001_CA_1_evaluation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
1,FOODS_1_001_CA_1_validation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
2,FOODS_1_001_CA_2_evaluation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
3,FOODS_1_001_CA_2_validation,1.95,1.37,1.37,1.29,1.40,1.96,1.37,1.37,1.37,...,1.29,1.40,1.88,1.95,1.95,1.91,2.14,1.96,1.37,1.37
4,FOODS_1_001_CA_3_evaluation,0.96,1.08,1.08,1.01,0.93,1.04,1.08,1.08,1.08,...,1.01,0.93,0.94,0.96,0.96,0.92,0.93,1.04,1.08,1.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,HOUSEHOLD_2_516_WI_1_validation,0.01,0.02,0.02,0.01,0.01,0.01,0.02,0.02,0.02,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02
60976,HOUSEHOLD_2_516_WI_2_evaluation,1.04,1.01,1.01,0.99,1.08,0.79,1.01,1.01,1.01,...,0.99,1.08,0.78,1.04,1.04,1.03,1.19,0.79,1.01,1.01
60977,HOUSEHOLD_2_516_WI_2_validation,0.05,0.16,0.16,0.12,0.03,0.03,0.16,0.16,0.16,...,0.12,0.03,0.03,0.05,0.05,0.05,0.01,0.03,0.16,0.16
60978,HOUSEHOLD_2_516_WI_3_evaluation,1.04,1.01,1.01,0.99,1.08,0.79,1.01,1.01,1.01,...,0.99,1.08,0.78,1.04,1.04,1.03,1.19,0.79,1.01,1.01


In [95]:
# And finally... we have our submisison file...
csv_submit = sub_file.to_csv('submission.csv', index = False)